In [6]:
l=[[0,1,0,1],[0,1,1,-1]]

In [7]:
for i in range(len(l)):
    cur = l[i]
    last = cur[-1]
    if last !=0:
        temp = [x/last for x in cur]
        l[i] = temp

In [8]:
ret = []
for i in range(len(l)):
    for j in range(i+1,len(l)):
        first = l[i]
        second = l[j]
        if first[-1] !=0 and second[-1] != 0:
            temp = [first[i] - second[i] for i in range(len(first))]
            ret.append(temp)

In [9]:
ret= [x[:-1] for x in ret]

In [10]:
ret

[[0.0, 2.0, 1.0]]

In [11]:
import numpy as np

# linear case

In [12]:
thresholds = [1]

In [13]:
def generatePMatrix(m, pmatrix):
    acc_prod = list(map(lambda x: int(x), [np.prod(m[i+1:]) for i in range(len(m))]))
    l = np.sum(m)-1
    p = [() for _ in range(np.prod(m))]
    level = []

    for i in range(len(p)):
        total = i
        level = []
        for j in range(len(m)):
            current = int(total/acc_prod[j])
            total %= acc_prod[j]
            level.append(current)
        curr_p = []
        for i in range(len(level)):
            temp = [0 for _ in range(m[i])]
            temp[m[i]-1-level[i]] = 1
            curr_p+=temp
        pmatrix.append(curr_p)

In [14]:
pmatrix = []
generatePMatrix([2],pmatrix)

In [15]:
epsilon = [1]
nepsilon = [-1]
notheta = 1
theta = []
for i in range(notheta):
    temp = [0 for  _ in range(notheta)]
    temp[i] = 1
    theta.append(temp)

In [16]:
def diffByOneBit(a,b):
    # a< b and differ only one bit
    diff = abs(a-b)
    if (a|b == b) and bin(diff).count('1') == 1:
        return True
    return False

def generateOneBitCondition(m):
    
    counts = [int(np.log2(i)) for i in m]
    bitsNo = [sum(counts[i+1:]) for i in range(len(counts))]
    
    pairs = set()
    for i in range(len(m)):
        component = m[i]
        for j in range(component):
            for k in range(j+1,component):
                if diffByOneBit(j,k):
                    pairs.add((j<<bitsNo[i],k<<bitsNo[i]))
            
    return pairs

In [17]:
t = generateOneBitCondition([2])
onebit = []
#b - s >= epsilon
for pair in t:
    s = pmatrix[pair[0]]
    b = pmatrix[pair[1]]
    diff = [b[i] - s[i] for i in range(len(b))]
    onebit.append(diff)

In [18]:
# last term for the constant term
oneBitEps = []
for x in onebit:
    oneBitEps.append(x + [0 for i in range(notheta)] + nepsilon + [0])

In [19]:
oneBitEps

[[1, -1, 0, -1, 0]]

In [20]:
polyWithTheta = []
for x in pmatrix:
    for y in theta:
        y = [-t for t in y]
        polyWithTheta.append(x+y+[0] + [0])

In [21]:
polyWithTheta

[[0, 1, -1, 0, 0], [1, 0, -1, 0, 0]]

In [22]:
oneBitEps

[[1, -1, 0, -1, 0]]

In [23]:
lBoundEps = [[0,1,0,-1,0]]
thetaBoundEps = [[0,0,1,-1,0]]

In [24]:
sdr = [[1 for _ in range(len(polyWithTheta[0]))]]
sdr[0][-2] = 0
sdr[0][-1] = -1

In [25]:
sdr

[[1, 1, 1, 0, -1]]

In [26]:
matrix = polyWithTheta + oneBitEps + lBoundEps + thetaBoundEps + sdr 

In [27]:
# eliminate from first to the second last
ret = matrix.copy()
while len(ret[0]) > 2:
    newMatrix = []
    for x in ret:
        if x[0] == 0:
            newMatrix.append(x[1:])
    
    for i in range(len(ret)):
        for j in range(i+1, len(ret)):
            
            if ret[i][0] == 0 or ret[j][0] == 0:
                continue
                
            first = list(ret[i])
            second = list(ret[j])
            first = [x/ret[i][0] for x in first]
            second = [x/ret[j][0] for x in second]
            
            diff = [first[i] - second[i] for i in range(len(first))]
            
            newMatrix.append(diff[1:])
    
    # remove duplicate
    newMatrix = [tuple(x) for x in newMatrix]
    newMatrix = list(set(newMatrix))
    
    ret = newMatrix
    
    print(ret)

[(0, 1, -1, 0), (1, 0, -1, 0), (1, -1, 0, 0), (-1.0, -2.0, 0.0, 1.0), (1.0, -1.0, 1.0, 0.0), (-2.0, -1.0, -1.0, 1.0)]
[(-2.0, -1.0, 1.0), (1.5, -0.5, -0.5), (-3.0, 0.0, 1.0), (1.0, -2.0, 0.0), (1, -1, 0), (-1.5, -0.5, 0.5), (-1.5, 0.5, 0.5), (3.0, -1.0, -1.0), (0.0, -1.0, 0.0), (-0.5, -1.5, 0.5)]
[(-1.0, 0.0), (-2.5, 0.5), (-2.3333333333333335, 0.3333333333333333), (-2.6666666666666665, 0.6666666666666667), (-1.3333333333333333, 0.3333333333333333), (-1.6666666666666667, 0.3333333333333333), (-5.0, 1.0), (0.5, -0.16666666666666669), (0.8333333333333333, -0.16666666666666669), (0.16666666666666669, -0.16666666666666669), (-0.3333333333333333, 0.0), (0.6666666666666667, -0.3333333333333333), (-0.6666666666666666, 0.0), (0.0, 0.0), (0.3333333333333333, 0.0), (-0.6666666666666667, 0.3333333333333333), (0.6666666666666666, 0.0), (1.0, -0.3333333333333333), (1.5, -0.5), (1.6666666666666667, -0.3333333333333333), (-3.3333333333333335, 0.6666666666666667), (-3.0, 0.6666666666666667), (2.5, -0.

In [28]:
# every x, theta, epsilon, d variable and constant term are on the left only zero on the right
candidates = []
for pair in ret:
    if pair[0]:
        candidates.append(-pair[1]/pair[0])

In [29]:
min([x for x in candidates if x>0])

0.14285714285714285

In [30]:
# two output linear

pmatrix = []
generatePMatrix([2],pmatrix)

epsilon = [1]
nepsilon = [-1]
notheta = 2
theta = []
for i in range(notheta):
    temp = [0 for  _ in range(notheta)]
    temp[i] = 1
    theta.append(temp)
    
t = generateOneBitCondition([2])
onebit = []
#b - s >= epsilon
for pair in t:
    s = pmatrix[pair[0]]
    b = pmatrix[pair[1]]
    diff = [b[i] - s[i] for i in range(len(b))]
    onebit.append(diff)
    
# last term for the constant term
oneBitEps = []
for x in onebit:
    oneBitEps.append(x + [0 for i in range(notheta)] + nepsilon + [0])
    
polyWithTheta = []
for x in pmatrix:
    for y in theta:
        y = [-t for t in y]
        polyWithTheta.append(x+y+[0] + [0])
        
lBoundEps = [[0,1,0,0,-1,0]]

thetaBoundEps = [[0,0,1,0,-1,0],[0,0,0,1,-1,0]]
thetaOrder = [[0,0,1,-1,0,0]]

sdr = [[1 for _ in range(len(polyWithTheta[0]))]]
sdr[0][-2] = 0
sdr[0][-1] = -1

In [31]:
sdr

[[1, 1, 1, 1, 0, -1]]

In [32]:
oneBitEps

[[1, -1, 0, 0, -1, 0]]

In [33]:
polyWithTheta

[[0, 1, -1, 0, 0, 0],
 [0, 1, 0, -1, 0, 0],
 [1, 0, -1, 0, 0, 0],
 [1, 0, 0, -1, 0, 0]]

In [34]:
from fractions import Fraction
one = Fraction(1,1)
zero = Fraction(0,1)

In [35]:
# two output linear
from fractions import Fraction
one = Fraction(1,1)
zero = Fraction(0,1)
pmatrix = []
generatePMatrix([2],pmatrix)
pmatrix = [list(map(lambda x: Fraction(x), y)) for y in pmatrix]

epsilon = [one]
nepsilon = [-one]
notheta = 2
theta = []
for i in range(notheta):
    temp = [zero for  _ in range(notheta)]
    temp[i] = one
    theta.append(temp)
    
t = generateOneBitCondition([2])
onebit = []
#b - s >= epsilon
for pair in t:
    s = pmatrix[pair[0]]
    b = pmatrix[pair[1]]
    diff = [b[i] - s[i] for i in range(len(b))]
    onebit.append(diff)
    
# last term for the constant term
oneBitEps = []
for x in onebit:
    oneBitEps.append(x + [zero for i in range(notheta)] + nepsilon + [zero])
    
polyWithTheta = []
for x in pmatrix:
    for y in theta:
        y = [-t for t in y]
        polyWithTheta.append(x+y+[zero] + [zero])
        
lBoundEps = [[zero,one,zero,one,-one,zero]]

thetaBoundEps = [[zero,zero,one,zero,-one,zero],[zero,zero,zero,one,-one,zero]]
thetaOrder = [[zero,zero,one,-one,zero,zero]]

sdr = [[one for _ in range(len(polyWithTheta[0]))]]
sdr[0][-2] = zero
sdr[0][-1] = -one

In [36]:
matrix = polyWithTheta + oneBitEps + lBoundEps + thetaOrder + thetaBoundEps + sdr 

In [37]:
# eliminate from first to the second last
ret = matrix.copy()
while len(ret[0]) > 2:
    newMatrix = []
    for x in ret:
        if x[0] == zero:
            newMatrix.append(x[1:])
    
    for i in range(len(ret)):
        for j in range(i+1, len(ret)):
            
            if ret[i][0] == zero or ret[j][0] == 0:
                continue
                
            first = list(ret[i])
            second = list(ret[j])
            first = [x/ret[i][0] for x in first]
            second = [x/ret[j][0] for x in second]
            
            diff = [first[i] - second[i] for i in range(len(first))]
            
            newMatrix.append(diff[1:])
    
    # remove duplicate
    newMatrix = [tuple(x) for x in newMatrix]
    newMatrix = list(set(newMatrix))
    
    ret = newMatrix
    
    print(ret)

[(Fraction(1, 1), Fraction(-1, 1), Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)), (Fraction(1, 1), Fraction(0, 1), Fraction(-1, 1), Fraction(0, 1), Fraction(0, 1)), (Fraction(0, 1), Fraction(1, 1), Fraction(0, 1), Fraction(-1, 1), Fraction(0, 1)), (Fraction(0, 1), Fraction(1, 1), Fraction(-1, 1), Fraction(0, 1), Fraction(0, 1)), (Fraction(1, 1), Fraction(0, 1), Fraction(1, 1), Fraction(-1, 1), Fraction(0, 1)), (Fraction(1, 1), Fraction(-1, 1), Fraction(0, 1), Fraction(1, 1), Fraction(0, 1)), (Fraction(1, 1), Fraction(0, 1), Fraction(-1, 1), Fraction(1, 1), Fraction(0, 1)), (Fraction(0, 1), Fraction(-1, 1), Fraction(1, 1), Fraction(0, 1), Fraction(0, 1)), (Fraction(-1, 1), Fraction(-2, 1), Fraction(-1, 1), Fraction(0, 1), Fraction(1, 1)), (Fraction(-1, 1), Fraction(-1, 1), Fraction(-2, 1), Fraction(0, 1), Fraction(1, 1)), (Fraction(0, 1), Fraction(0, 1), Fraction(1, 1), Fraction(-1, 1), Fraction(0, 1)), (Fraction(-2, 1), Fraction(-1, 1), Fraction(-1, 1), Fraction(-1, 1), Fraction(1, 1

[(Fraction(7, 4), Fraction(-3, 4)), (Fraction(7, 4), Fraction(-1, 4)), (Fraction(5, 4), Fraction(-1, 4)), (Fraction(5, 4), Fraction(-3, 4)), (Fraction(13, 8), Fraction(-1, 4)), (Fraction(15, 8), Fraction(-3, 4)), (Fraction(11, 8), Fraction(-1, 2)), (Fraction(9, 8), Fraction(-1, 4)), (Fraction(9, 8), Fraction(-3, 4)), (Fraction(5, 4), Fraction(0, 1)), (Fraction(5, 3), Fraction(-2, 3)), (Fraction(23, 12), Fraction(-5, 12)), (Fraction(13, 6), Fraction(-2, 3)), (Fraction(-11, 2), Fraction(3, 2)), (Fraction(-11, 2), Fraction(5, 4)), (Fraction(5, 1), Fraction(-3, 2)), (Fraction(11, 2), Fraction(-3, 2)), (Fraction(-5, 1), Fraction(3, 2)), (Fraction(-5, 1), Fraction(1, 1)), (Fraction(-21, 4), Fraction(3, 2)), (Fraction(-5, 1), Fraction(5, 4)), (Fraction(23, 4), Fraction(-5, 4)), (Fraction(-3, 8), Fraction(1, 4)), (Fraction(3, 8), Fraction(-1, 4)), (Fraction(1, 2), Fraction(1, 4)), (Fraction(3, 4), Fraction(1, 4)), (Fraction(-1, 4), Fraction(1, 2)), (Fraction(-3, 4), Fraction(1, 2)), (Fraction(

In [38]:
# every x, theta, epsilon, d variable and constant term are on the left only zero on the right
candidates = []
for pair in ret:
    if pair[0]:
        candidates.append(-pair[1]/pair[0])
min([x for x in candidates if x>0])

Fraction(1, 23)

# log linear xy

In [39]:
# two output linear

pmatrix = []
generatePMatrix([2,2],pmatrix)

epsilon = [1]
nepsilon = [-1]
notheta = 1
d = [1]
nd = [-1]
theta = []
for i in range(notheta):
    temp = [0 for  _ in range(notheta)]
    temp[i] = 1
    theta.append(temp)
    
t = generateOneBitCondition([2,2])
onebit = []
#b - s >= epsilon
for pair in t:
    s = pmatrix[pair[0]]
    b = pmatrix[pair[1]]
    diff = [b[i] - s[i] for i in range(len(b))]
    onebit.append(diff)
    
# last term for the constant term
oneBitEps = []
for x in onebit:
    oneBitEps.append(x + [0 for i in range(notheta)] + nepsilon +[0] + [0])
    
polyWithTheta = []
for x in pmatrix:
    for y in theta:
        y = [-t for t in y]
        polyWithTheta.append(x+y+ [0,0] + [0])
        
lBoundD = [[0,1,0,0,0,0,-1,0],[0,0,0,1,0,0,-1,0]]
thetaBoundD =[[0,0,0,0,1,0,-1,0]]

sdr = [[1 for _ in range(len(polyWithTheta[0]))]]
sdr[0][-3:-1] = [0,0]
sdr[0][-1] = 0

In [40]:
matrix = polyWithTheta + oneBitEps + lBoundD + thetaBoundD + sdr 

In [41]:
# eliminate from first to the second last
# get bound for epsilon first
ret = matrix.copy()
while len(ret[0]) > 3:
    newMatrix = []
    for x in ret:
        if x[0] == 0:
            newMatrix.append(x[1:])
    
    for i in range(len(ret)):
        for j in range(i+1, len(ret)):
            
            if ret[i][0] == 0 or ret[j][0] == 0:
                continue
                
            first = list(ret[i])
            second = list(ret[j])
            first = [x/ret[i][0] for x in first]
            second = [x/ret[j][0] for x in second]
            
            diff = [first[i] - second[i] for i in range(len(first))]
            
            newMatrix.append(diff[1:])
    
    # remove duplicate
    newMatrix = [tuple(x) for x in newMatrix]
    newMatrix = list(set(newMatrix))
    
    ret = newMatrix
    
    #print(ret)

In [42]:
candidates = []
for pair in ret:
    if pair[1]:
        candidates.append(-pair[0]/pair[1])
max([x for x in candidates if x>0])

3.833333333333333

# log linear (x+y)z

In [47]:
# two outputs

pmatrix = []
generatePMatrix([4,2],pmatrix)

epsilon = [1]
nepsilon = [-1]
notheta = 2
d = [1]
nd = [-1]
theta = []
for i in range(notheta):
    temp = [0 for  _ in range(notheta)]
    temp[i] = 1
    theta.append(temp)
    
t = generateOneBitCondition([4,2])
onebit = []
#b - s >= epsilon
for pair in t:
    s = pmatrix[pair[0]]
    b = pmatrix[pair[1]]
    diff = [b[i] - s[i] for i in range(len(b))]
    onebit.append(diff)
    
# last term for the constant term
oneBitEps = []
for x in onebit:
    oneBitEps.append(x + [0 for i in range(notheta)] + nepsilon +[0] + [0])
    
polyWithTheta = []
for x in pmatrix:
    for y in theta:
        y = [-t for t in y]
        polyWithTheta.append(x+y+ [0,0] + [0])
        
lBoundD = [[0,0,0,1,0,0,0,0,0,-1,0],[0,0,0,0,0,1,0,0,0,-1,0]]

thetaBoundD =[[0,0,0,0,0,0,1,0,0,-1,0],[0,0,0,0,0,0,0,1,0,-1,0]]

thetaOrder = [[0,0,0,0,0,0,0,0,0,0,0]]

sdr = [[1 for _ in range(len(polyWithTheta[0]))]]
sdr[0][-3:] = [0,0,0]

In [48]:
matrix = polyWithTheta + oneBitEps + lBoundD + thetaBoundD + sdr 

In [50]:
matrix.__len__()

26

In [56]:
# eliminate from first to the second last
# get bound for epsilon first
ret = matrix.copy()
while len(ret[0]) > 3:
    print(len(ret[0]))
    newMatrix = []
    for x in ret:
        if x[0] == 0:
            newMatrix.append(x[1:])
    
    for i in range(len(ret)):
        for j in range(i+1, len(ret)):
            
            if ret[i][0] == 0 or ret[j][0] == 0:
                continue
                
            first = list(ret[i])
            second = list(ret[j])
            firstLC = first[0]
            secondLC = second[0]
            first = [x*secondLC for x in first]
            second = [x*firstLC for x in second]
            
            diff = [first[i] - second[i] for i in range(len(first))]
            for ele in diff:
                if ele!=0:
                    newMatrix.append(diff[1:])
                    break    
            
    # remove duplicate
    newMatrix = [tuple(x) for x in newMatrix]
    newMatrix = list(set(newMatrix))
    
    ret = newMatrix
    
    #print(ret)

11
10
9
8


KeyboardInterrupt: 

In [53]:
# two outputs

pmatrix = []
generatePMatrix([4,2],pmatrix)
pmatrix = [list(map(lambda x: Fraction(x), y)) for y in pmatrix]

epsilon = [one]
nepsilon = [-one]
notheta = 2
d = [one]
nd = [-one]
theta = []
for i in range(notheta):
    temp = [zero for  _ in range(notheta)]
    temp[i] = one
    theta.append(temp)
    
t = generateOneBitCondition([4,2])
onebit = []
#b - s >= epsilon
for pair in t:
    s = pmatrix[pair[0]]
    b = pmatrix[pair[1]]
    diff = [b[i] - s[i] for i in range(len(b))]
    onebit.append(diff)
    
# last term for the constant term
oneBitEps = []
for x in onebit:
    oneBitEps.append(x + [zero for i in range(notheta)] + nepsilon +[zero] + [zero])
    
polyWithTheta = []
for x in pmatrix:
    for y in theta:
        y = [-t for t in y]
        polyWithTheta.append(x+y+ [zero,zero] + [zero])
        
lBoundD = [[zero,zero,zero,one,zero,zero,zero,zero,zero,-one,zero],[zero,zero,zero,zero,zero,one,zero,zero,zero,-one,zero]]

thetaBoundD =[[zero,zero,zero,zero,zero,zero,one,zero,zero,-one,zero],[zero,zero,zero,zero,zero,zero,zero,one,zero,-one,zero]]

thetaOrder = [[zero,zero,zero,zero,zero,zero,zero,zero,zero,zero,zero]]

sdr = [[one for _ in range(len(polyWithTheta[0]))]]
sdr[0][-3:] = [zero,zero,zero]

In [54]:
matrix = polyWithTheta + oneBitEps + lBoundD + thetaBoundD + sdr 

In [55]:
matrix

[[Fraction(0, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(1, 1),
  Fraction(0, 1),
  Fraction(1, 1),
  Fraction(-1, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(0, 1)],
 [Fraction(0, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(1, 1),
  Fraction(0, 1),
  Fraction(1, 1),
  Fraction(0, 1),
  Fraction(-1, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(0, 1)],
 [Fraction(0, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(1, 1),
  Fraction(1, 1),
  Fraction(0, 1),
  Fraction(-1, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(0, 1)],
 [Fraction(0, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(1, 1),
  Fraction(1, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(-1, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(0, 1)],
 [Fraction(0, 1),
  Fraction(0, 1),
  Fraction(1, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(1, 1),
  Fraction(-1, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(0, 1),
  Fraction(0, 1)],


In [59]:
# eliminate from first to the second last
# get bound for epsilon first
ret = matrix.copy()
while len(ret[0]) > 3:
    print(len(ret))
    newMatrix = []
    for x in ret:
        if x[0] == 0:
            newMatrix.append(x[1:])
    
    for i in range(len(ret)):
        for j in range(i+1, len(ret)):
            
            if ret[i][0] == 0 or ret[j][0] == 0:
                continue
                
            first = list(ret[i])
            second = list(ret[j])
            first = [x/ret[i][0] for x in first]
            second = [x/ret[j][0] for x in second]
            
            diff = [first[i] - second[i] for i in range(len(first))]
            first = diff[1]
            if first != 0:
                diff = [x/first for x in diff]
            
            newMatrix.append(diff[1:])
    
    # remove duplicate
    newMatrix = [tuple(x) for x in newMatrix]
    newMatrix = list(set(newMatrix))
    
    ret = newMatrix
    
    #print(ret)

26
38
83
484
12237


KeyboardInterrupt: 

In [1]:
for i in range(1,3):
    for i in range(1,4):
        print(i)

1
2
3
1
2
3
